In [2]:
import pandas as pd

In [8]:
usRoutes_csv = pd.read_csv('../data/usRoutes.csv')
airports_csv = pd.read_csv('../data/openFlightsRaw/airports.csv')

nullVal = r"\N"
usRoutes = usRoutes_csv[usRoutes_csv['seconds'] != nullVal]

airports = airports_csv[['AIRPT_ID', 'IATA']]

usRoutes = usRoutes.merge(airports, how="left", left_on='SRC_AIRPT_ID', right_on='AIRPT_ID')
usRoutes = usRoutes.merge(airports, how="left", left_on='DESTIN_AIRPT_ID', right_on='AIRPT_ID')

usRoutes = usRoutes.rename(columns={'IATA_x': 'SRCIATA', 'IATA_y': 'DESTINIATA'})

usRoutes = usRoutes.drop(['Unnamed: 0', 'og_index', 'AIRPT_ID_x', 'AIRPT_ID_y'], axis=1)

usRoutes = usRoutes.drop_duplicates(keep='last')

usRoutes.to_csv('../data/usRoutesClean.csv')

In [4]:
flightSamples_csv = pd.read_csv('../data/kaggleRaw/flightSamples.csv')

In [22]:
flightSamples = flightSamples_csv[['Fly_date', 'Flights', 'Origin_airport', 'Destination_airport']]

flightSamples = flightSamples.merge(usRoutes, how="left", left_on=['Origin_airport', 'Destination_airport'], right_on=['SRCIATA', 'DESTINIATA'])

flightSamples = flightSamples.dropna()
flightSamples = flightSamples.reset_index()
flightSamples = flightSamples.drop(['index', 'og_index', 'SRC_AIRPT_ID', 'DESTIN_AIRPT_ID', 'Origin_airport', 'Destination_airport'], axis=1)


In [24]:
flightSamples
flightSamples.to_csv('../data/flightSamplesClean.csv')

In [26]:
flightSamples

,Fly_date,Flights,SRCNAME,DESTINNAME,seconds,meters,SRCIATA,DESTINIATA
0,1990-02-01,1,Seattle Tacoma International Airport,Roberts Field,141650,3481926,SEA,RDM
1,1990-01-01,23,Seattle Tacoma International Airport,Roberts Field,141650,3481926,SEA,RDM
2,1990-02-01,20,Seattle Tacoma International Airport,Roberts Field,141650,3481926,SEA,RDM
3,1990-03-01,22,Seattle Tacoma International Airport,Roberts Field,141650,3481926,SEA,RDM
4,1990-04-01,1,Seattle Tacoma International Airport,Roberts Field,141650,3481926,SEA,RDM
...,...,...,...,...,...,...,...,...
6601693,2009-05-01,70,St Louis Lambert International Airport,Waynesville-St. Robert Regional Forney field,14296,437620,STL,TBN
6601694,2009-06-01,69,St Louis Lambert International Airport,Waynesville-St. Robert Regional Forney field,14296,437620,STL,TBN
6601695,2009-02-01,51,St Louis Lambert International Airport,Waynesville-St. Robert Regional Forney field,14296,437620,STL,TBN
6601696,2009-11-01,54,St Louis Lambert International Airport,Waynesville-St. Robert Regional Forney field,14296,437620,STL,TBN


data i have, how many flights happen betweeen two places
distance and seconds between two places

turn lat/long data into zones through clustering with kmeans, 

could do some prediction based on the timseries and frequency
Then i could check to see if there is something in the same zone with distance or seconds close or less, need to factor in the predicted frequency because it means availability of a trip

In [39]:
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px

In [155]:
airportsData = airports_csv
airportsData = airportsData[airportsData['CTRY'] == 'United States']

airportsCoords = airportsData[['LAT', 'LONG']]
airportsCoords = airportsCoords.to_numpy()

In [82]:
inertiaList = []
kList = range(1, 50)

for k in kList:
    kMeans = KMeans(n_clusters = k, random_state = 0, n_init = 10).fit(airportsCoords)
    inertiaList.append(kMeans.inertia_)

In [83]:
inertiaDF = pd.DataFrame(inertiaList, columns=['Inertias'])
fig = px.scatter(inertiaDF, y="Inertias")
fig.show()

In [84]:
inertiaDF['InertiasSquared'] = np.sqrt(inertiaDF['Inertias'])
fig = px.scatter(inertiaDF, y="InertiasSquared")
fig.show()

In [158]:
airportsData['cluster'] = KMeans(n_clusters = 22, random_state = 0, n_init = 10).fit_predict(airportsCoords)

C:\Users\thoma\AppData\Local\Temp\ipykernel_31608\1904615938.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [219]:
airportsData['clusterColor'] = airportsData['cluster'].map(pd.Series(px.colors.qualitative.Dark24))

C:\Users\thoma\AppData\Local\Temp\ipykernel_31608\3262556980.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [220]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    name = 'string',
    lat = airportsData['LAT'].tolist(),
    lon = airportsData['LONG'].tolist(), 
    mode = 'markers',
    marker = dict(
        size = 3,
        color = airportsData['clusterColor'].tolist(),
    ),
))

fig.update_layout(
    title_text='All US Airports have Incoming and Outgoing Traffic',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [ ]:
from sklearn.cluster import DBSCAN

In [190]:
itemList = []
epsilonList = np.arange(0.005, 1, 0.005)

for e in epsilonList:
    dbscan = DBSCAN(eps = e, min_samples = 10, metric = 'haversine').fit((np.radians(airportsCoords)))
    itemList.append(len(np.unique(dbscan.labels_)))

In [191]:
itemDF = pd.DataFrame({'Epislon': epsilonList, 'Clusters': itemList})
fig = px.scatter(itemDF, y = 'Clusters', x = 'Epislon')
fig.show()

In [186]:
itemList = []
epsilonList = np.arange(0.00001, 0.055, 0.00001)

for e in epsilonList:
    dbscan = DBSCAN(eps = e, min_samples = 10, metric = 'haversine').fit((np.radians(airportsCoords)))
    itemList.append(len(np.unique(dbscan.labels_)))

In [187]:
itemDF = pd.DataFrame({'Epislon': epsilonList, 'Clusters': itemList})
fig = px.scatter(itemDF, y = 'Clusters', x = 'Epislon')
fig.show()

In [ ]:
1/6371
https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

0.0001569612305760477

In [231]:
airportsData['densityCluster'] = DBSCAN(eps = 0.01569, min_samples = 10, metric = 'haversine').fit_predict((np.radians(airportsCoords)))

C:\Users\thoma\AppData\Local\Temp\ipykernel_31608\1314355970.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Keep in mind -1 represents noise, and doesn't end up getting clustered
airportsData['densityCluster'] = airportsData['densityCluster'] + 1
airportsData['densityClusterColor'] = airportsData['densityCluster'].map(pd.Series(px.colors.qualitative.Dark24))

C:\Users\thoma\AppData\Local\Temp\ipykernel_31608\4044834508.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_31608\4044834508.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [233]:
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    name = 'string',
    lat = airportsData['LAT'].tolist(),
    lon = airportsData['LONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 3,
        color = airportsData['densityClusterColor'].tolist()
    )
))

fig.update_layout(
    title_text='All US Airports have Incoming and Outgoing Traffic',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [ ]:
airportsDataDBScan = airportsData

airportsDataDBScan = airportsDataDBScan[airportsDataDBScan['densityCluster'] != 0]

In [244]:
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    name = 'string',
    lat = airportsDataDBScan['LAT'].tolist(),
    lon = airportsDataDBScan['LONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 3,
        color = airportsDataDBScan['densityClusterColor'].tolist()
    )
))

fig.update_layout(
    title_text='All US Airports have Incoming and Outgoing Traffic',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [ ]:
washingtonCluster = usRoutes[usRoutes['DESTIN_AIRPT_ID'].isin(list(airportsDataDBScan[airportsDataDBScan['densityCluster'] == 6]['AIRPT_ID']))]
washingtonCluster[washingtonCluster['DESTIN_AIRPT_ID'] != 3577]

#Bellingham International Airport, can accept flights to go to Seattle Tacoma International Airport

,og_index,SRC_AIRPT_ID,SRCNAME,DESTIN_AIRPT_ID,DESTINNAME,seconds,meters,SRCIATA,DESTINIATA
2297,11358,3728,Daniel K Inouye International Airport,3777,Bellingham International Airport,69184,2174892,HNL,BLI
2505,11616,3577,Seattle Tacoma International Airport,3777,Bellingham International Airport,110692,3457980,SEA,BLI
5370,28478,6505,Phoenix-Mesa-Gateway Airport,3777,Bellingham International Airport,63057,1949107,AZA,BLI
5495,28608,3877,McCarran International Airport,3777,Bellingham International Airport,73630,2302614,LAS,BLI
5534,28647,3484,Los Angeles International Airport,3777,Bellingham International Airport,66234,2023907,LAX,BLI
5561,28679,3453,Metropolitan Oakland International Airport,3777,Bellingham International Airport,79387,2491829,OAK,BLI
5639,28760,3731,San Diego International Airport,3777,Bellingham International Airport,51742,1625762,SAN,BLI
